In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
%config Completer.use_jedi = False  # to make autocompletion faster

In [ ]:
import tensorflow as tf
import pandas as pd
import gc

In [ ]:
from webtraffic_utils import *

In [ ]:
df_train = pd.read_csv("train_2.csv.zip", header=0).set_index("Page").fillna(0).astype(np.int32)
#df_train = pd.read_csv("train_2.csv.zip", header=0).dropna().set_index("Page").astype(np.int32).reset_index()

In [ ]:
access = pd.Series(["_".join(ii.split("_")[-2:]) for ii in df_train.index], index=df_train.index, name="access")
access.value_counts()

In [ ]:
project = pd.Series([ii.split("_")[-3] for ii in df_train.index], index=df_train.index, name="project")
project.value_counts()

### Spectral analysis

In [ ]:
page_ex = 'Acier_inoxydable_fr.wikipedia.org_desktop_all-agents'
x = df_train.loc[page_ex].values
#x = df_train.iloc[0].values[1:]
xc = x-np.mean(x, dtype=np.float)
xn = xc / np.linalg.norm(xc) 
#np.linalg.norm([1,1])
fn = np.fft.rfft(xn)
tones = np.argwhere(np.abs(fn)**2 >10) / len(x)

f,ax = plt.subplots()
ax.plot(np.linspace(0,0.5,num=len(fn)),np.abs(fn)**2)
ax.set_yscale("log")
ax.set_ylim(bottom=1e-1)
ax.grid()
np.sum(xn**2),np.mean(np.abs(fn)**2),1./7,1./365
tones.reshape(-1,).shape

In [ ]:
def find_tones(x):
    xc = x-np.mean(x, dtype=np.float)
    xn = xc / (np.linalg.norm(xc) + 1e-16)
    #np.linalg.norm([1,1])
    fn = np.fft.rfft(xn)
    tones = np.argwhere(np.abs(fn)**2 >10) / len(x)
    return list(tones.reshape(-1,))

tones = df_train.apply(lambda x: find_tones(x.values), axis=1).rename("tones")

In [ ]:
(tones.apply(len)>0).mean()

In [ ]:
weekly_tone = tones.apply(lambda x: (np.abs(np.array(x)-1./7.)<1e-2).any())
weekly_tone.mean()

In [ ]:
%matplotlib
f,ax = plt.subplots()
tones.explode().dropna().hist(ax=ax, bins = 100)
ax.set_yscale("log")

##### median

In [ ]:
s_median = pd.Series(np.median(df_train.values[:,:-62], axis=1), index=df_train.index, name="median")

f, ax = plt.subplots()
#ax.hist(np.log1p(s_median), bins=50)
#ax.hist(s_median, cumulative=True, bins=100, density=True)
#ax.boxplot(np.random.randn(100))
ax.boxplot(s_median.dropna(), showfliers=False)
ax.grid()
ax.set_title("median traffic distribution")
plt.show()

###### null/zero values

In [ ]:
last_values_null = pd.Series(np.mean(df_train.values[:,-62-200:-62] == 0, axis=1), index=df_train.index, name="last_values_null")

In [ ]:
df_feats = pd.concat([access, project, tones, s_median, last_values_null], axis=1)

In [ ]:
df_feats.to_csv("features_computed.csv.zip")